#PROGRAMACIÓN CONCURRENTE

La programación concurrente con su forma de estructurar procesos independientes ha logrado que las tareas sean mas rapidas y eficientes, pero, no sin pagar un pequeño costo. Estos procesos avanzan casi al mismo tiempo y son capaces de comunicarse entre ellos para sacar adelante tareas complejas, pero sin mecanismos que aseguren una buena sincronizacion entre estos, lo que deberia ser mas rapido ahora es más lento y hay veces en las que no termina incluso.
A continuación mostraremos algunos aspectos y problemas con los que se debe ser cuidadoso al tyrabajar con programas concurrentes.

##NO DETERMINISMO
En programas concurrentes, se presenta la cualidad de tener software que no van a dar siempre la misma salida, sea o no igual su entrada. Lo anterior se debe en si a la naturaleza misma de la porgramación concurrente, adicional a características propias del lenguaje usado.

En general en la concurrencia, los hilos independientes de ejecución, van siendo despachados por el procesador, siguiendo las intrucciones que el código del hilo describe. Dado a que varios hilos solicitan ser despachados por el procesador en periodos intercalados, se puede dar a que un hilo que entró de primeras no sea el primero que sale, ni que el primero que entró sea el último en salir, sino que su orden depende de la cantidad de instrucciones, el tipo de instrucciones, los recursos, las respuestas, sincronizaciones y demás elementos que condicionan el orden en el que se van dando los resultados.


### EJEMPLO ADA
En el siguiente ejemplo de código Ada vamos a ver cómo se puede dar el no determinismo en un caso muy sencillo.

Primero veremos como se estructura la concurrencia en Ada.

En la parte inicial se indica que usaremos la salida de texto en consola, para ello hacemos:

In [0]:

with Ada.Text_Io;
use Ada.Text_Io;

A continuación ponemos nuestro procedimiento principal, llamado *Hello*  que contendrá 3 tareas. Hay que tener en cuenta que el procedimiento en sí mismo es también una tarea. En Ada las tareas son llamadas Task.

In [0]:

with Ada.Text_Io;
use Ada.Text_Io;
procedure Hello is
  
  task Tarea1;
  task Tarea2;
  task Tarea3;

  task body Tarea1 is
  begin
    
      Put_Line("Hilo 1 ejecutado");
    
  end Tarea1;

  task body Tarea2 is
  begin
    
      Put_Line("Hilo 2 ejecutado");
    
  end Tarea2;

   task body Tarea3 is
  begin
    
      Put_Line("Hilo 3 ejecutado");
    
  end Tarea3;

begin
  -- En este momento comienzan a ejecutarse ambas tareas.
  Put_Line("Finalizacion procedimiento principal");
end Hello;


Cada tarea, se define en dos momentos: la primera declarandolo (task Tarea; ), la segunda es el cuerpo, donde se define que es lo que hará la tarea.

###Ejecución de las tareas y su orden
El orden de las tareas se establece de acuerdo al orden en el que se declararon. Las tareas son llamadas a ser ejecutadas apenas al ejecutarse llegar a *Begin*  en el procedimiento Hello.
Las tareas son llamadas a ser ejecutadas, para luego ser ejecutadas.

###Momento de no determinismo
Cuando las tareas son llamadas a ser ejecutadas en el *Begin* , en teoría se mostraría en pantalla HIlo 1 ejecutado, Hilo 2 ejecutado, Hilo 3 ejecutado, en ese mismo orden, hasta acabar el programa imprimiendo "Finalización procedimiento principal".

Si se ejecuta de forma sucesiva el mismo programa, nuestro ejemplo, se podrá notar que en algunos casos ese orden no se da, ésto se debe a que **** **la operación imprimir en pantalla "Puts" no es atómica** es decir, la operación "puts" se puede dividir, y es devido a que es un compuesto de operaciones de llamado del sistema, es por ello que un hilo puede que llegue y espere por un corto tiempo a que sea devuelto su impresión en pantalla, mientras en ese mismo momento otro hilo llega y obtiene más rápido que el primer hilo que llegó, su impresión en pantalla, alterando el orden de ejecución.

##PROBLEMAS EN LA CONCURRENCIA

## Violación de la exclusión mutua
Ocurre cuando varios procesos intentan ejecutar una sección critica del código al mismo tiempo y lo logran con resultados indeseados. En este ejemplo de Java tenemos dos hilos que acceden a un elemento contador y activan su función de incremento, cada hilo hace esto 500 mil veces, y como el objeto inicia en 0 se deberia acabar con 1 millon en el contador, pero no es asi.
Cada vez que se ejecuta el programa se obtienen resultados distintos.

In [0]:
 public class Counting {
  public static void main(String[] args) throws InterruptedException {
  class Counter {
    int counter = 0;
    public void increment() { counter++; }
    public int get() { return counter; }
  }
  final Counter counter = new Counter();

  class CountingThread extends Thread {
    public void run() {
    for (int x = 0; x < 500000; x++) {
      counter.increment();
      }
    }
  }

  CountingThread t1 = new CountingThread();
  CountingThread t2 = new CountingThread();
  t1.start(); t2.start();
  t1.join(); t2.join();
  System.out.println(counter.get());//deberia dar 1000000
  }
 }
    

##Lockout o aplazamiento indefinido
Éste problema se basa en que, dados dos o más procesos o hilos, corriendo concurrentemente, uno de esos hilos se quede esperando indefinidamente a que se libere un recurso que necesita y que está siendo usado por otro hilo, o cuando el hilo que se queda en espera nunca es llamado y se mantiene en espera, sin ser finalmente nunca llamado a ser ejecutado.

Para ejemplificar mejor el problema, mostraremos un ejemplo hecho en Ada, donde dos tareas (hilos) contienen un ciclo. Un hilo hace el llamado al otro.

El ejemplo muestra el orden en el que se come un perro caliente: primero se hace el perro caliente, luego se pone en un plato, se le pone mostaza, come hasta decir que no se quiere más.

El ejemplo es el siguiente:

In [0]:
with Ada.Text_IO;
use Ada.Text_IO;
--rendezvous asimetrico llamada y espera de tarea
procedure HotDog is
   task Gourmet is
      entry Make_A_Hot_Dog;
   end Gourmet;
   task body Gourmet is
   begin
      Put_Line("Voy a preparar un perro caliente");
      for Index in 1..5 loop --si se cambia el 4 por un 5 se entrará en inanicion porque el task main del procedure hotdog no recibira la llamada 
         accept Make_A_Hot_Dog do
            delay 0.8;
            Put("Pongo el perro caliente en la mesa ");
            Put_Line(" y le pongo mostaza");
         end Make_A_Hot_Dog;
      end loop;
      Put_Line("ya no quiero mas perros calientes");
   end Gourmet;
begin
   for Index in 1..4 loop
      Gourmet.Make_A_Hot_Dog;
      delay 0.1;
      Put_Line("como lo que me queda de perro caliente");
      New_Line;
   end loop;
   Put_Line("ya estoy lleno");
end HotDog;

Podemos ver que se tiene un procedimiento llamado HoDog, el cual contiene una tarea: Gourmet. En definitiva recordemos que tenemos dos hilos, añadiendo que el procedimiento en si (HotDog) es un hilo.

En la declaración de la tarea Gourmet, se puede ver que hay una sentencia *entry* , que es la encargada de recibir como entrada. Es ésta la sentencia que es llamada para que el hilo Gourmet sea ejecutado.

En el cuerpo de la tarea Gourmet, se tiene un ciclo el cual contiene una sentencia *accept*, ésta sentencia define el momento en el que el hilo se detiene a esperar ser ejecutado. Es decir, otro hilo que quiera ejecutar el hilo Gourmet, debe hacer el entry de Make_A_Hot_Dog, y el hilo Gourmet va a ejecutarse en accept, donde se ejecuta.

Nótese que como la sentencia accept está dentro de un ciclo, el hilo va a ejecutarse y al volver a llegar a accept, se detendrá hasta que sea llamado de nuevo por otro hilo.

In [0]:
begin
      Put_Line("Voy a preparar un perro caliente");
      for Index in 1..5  
         accept Make_A_Hot_Dog do
            delay 0.8;
            Put("Pongo el perro caliente en la mesa ");
            Put_Line(" y le pongo mostaza");
         end Make_A_Hot_Dog;
      end loop;
      Put_Line("ya no quiero mas perros calientes");
   end Gourmet;

El hilo del procedimiento hace el llamado a ejecución del hilo (tarea) en un ciclo hasta 4:

In [0]:
for Index in 1..4 loop
      Gourmet.Make_A_Hot_Dog;
      delay 0.1;
      Put_Line("como lo que me queda de perro caliente");
      New_Line;
   end loop;

Entonces, el hilo del procedimiento HotDog llama al hilo Gourmet, el hilo Gourmet se ejecuta dentro de su ciclo hasta 5, luego el hilo HotDog vuelve a llamar a Gourmet y así continúa en su ciclo hasta 4.

###Se presenta el lockout

Note que el ciclo de HotDog va hasta 4 y el de Gourmet hasta 5, adicionalmente, recuerde que es el hilo de HoDog el que hace el llamado a ejecución de Gourmet. Debido a que el hilo HotDog se ejecutará 4 veces, va a dejar uno de menos respecto al de Gourmet en ejecutar, es decir, en el momento en el que el ciclo HotDog termina el ciclo en su 4ta iteración, en el hilo Gourmet se va a detener la ejecución en la sentencia accept hasta que sea llamado de nuevo, pero debido a que el hilo que hace su llamado, el hilo HotDog, no hará más llamados, **el hilo Gourmet va a quedarse esperando indefinidamente a ser llamado en la sentencia accept**.

##Deadlock o abrazo mortal
Éste problema se basa en el caso en el que, estando dos hillos corriendo concurrentemente, uno de ellos se queda esperando un evento que nunca ocurrirá.

Se muestra el ejemplo en el siguiente código Ada:

In [0]:
                                    -- Chapter 28 - Program 1
with Ada.Text_IO;
use Ada.Text_IO;
  --PROGRAMA QUE PROVOCA UN DEADLOCK https://perso.telecom-paristech.fr/pautet/Ada95/chap28.htm
procedure Meals1 is

   HOURS : constant := 1;
   type PERSON is (BILL, JOHN);

   package Enum_IO is new Ada.Text_IO.Enumeration_IO(PERSON);
   use Enum_IO;

   task Bills_Day;
   task Johns_Day;

   task Restaurant is
      entry Eat_A_Meal(Customer : PERSON);
   end Restaurant;

   task Burger_Boy is
      entry Eat_A_Meal(Customer : PERSON);
   end Burger_Boy;

   task body Bills_Day is
      My_Name : PERSON := BILL;
   begin
      delay 1.0 * HOURS;
      Restaurant.Eat_A_Meal(My_Name);
      delay 1.0 * HOURS;
      Restaurant.Eat_A_Meal(My_Name);
      delay 1.0 * HOURS;
      Restaurant.Eat_A_Meal(My_Name);
   end Bills_Day;

   task body Johns_Day is
      My_Name : PERSON := JOHN;
   begin
      delay 0.4 * HOURS;
      Restaurant.Eat_A_Meal(My_Name);
      delay 0.4 * HOURS;
      Restaurant.Eat_A_Meal(My_Name);
      delay 4.0 * HOURS;
      Restaurant.Eat_A_Meal(My_Name);
   end Johns_Day;

   task body Restaurant is
   begin
      loop
         accept Eat_A_Meal(Customer : PERSON) do
            Put(Customer);
            Put_Line(" esta ordenando en el restaurante");
            delay 0.5 * HOURS;
            Put(Customer);
            Put_Line(" esta comiendo en el restaurante");
            delay 0.5 * HOURS;
         end Eat_A_Meal;
      end loop;
   end Restaurant;

   task body Burger_Boy is
   begin
      loop
         accept Eat_A_Meal(Customer : PERSON) do
            Put(Customer);
            Put_Line(" esta ordenando en el McDonalds");
            delay 0.1 * HOURS;
            Put(Customer);
            Put_Line(" esta comiendo en el McDonalds");
            delay 0.1 * HOURS;
         end Eat_A_Meal;
      end loop;
   end Burger_Boy;

begin
   null;
end Meals1;

El programa consiste de 4 tareas, cada una para descirbir Bill's day, y John's day, en cada uno ellos van y comen en un restaurante y en McDonalds. Los delays definen que se tiene un tiempo de media hora (variable HOURS) esperando en restaurante para pedir y media para comer, uno de llos come más rápido que el otro.  A la final tendrán que haber comida 3 veces.

###Ocurre el deadlock
SI se hace una inspección detallada, se notará que las tareas Bills_day y Johns_day se ejecutan secuencialmente hasta que terminan, y no hacen llamadas adicionales. SIn embargo Restaurante y Burger_Boy tienen llamados infinitos, por lo que **el deadlock se presenta porque las tareas Bills_day y Johns_day ya terminaron y no harán más llamados, el sistema ve que no hay llamadas en lso ciclos infinitos de las otras tareas, porque lo que en esos ciclos se esperará por algo que no pasará**

##MÉTODOS DE SINCRONIZACIÓN

##Semáforos
Es el método de sincronización más simple, la intención es que un proceso bloqué la sección crítica para que otro no pueda ingresar hasta que el primero termine, las funciones elementales para implementar un semáforo en c++ son:
* sem_open: crea y habilita el semaforo
* sem_wait: para bloquear el semaforo
* sem_post: para desbloquear el semaforo
* sem_close: dehabilita el semaforo
* sem_unlink: remueve el semaforo

En el ejemplo se utiliza para que cada hilo que ingrese ejecute el código de la función en su totalidad sin interferencia de otro hilo, como se muestra a continuación:

 * hello world, upsi 2 ḧello  *--- 
 * hello world, upsi 1 ḧello  *--- 
 * hello world, upsi 3 ḧello  *---
 
 de no utilizarlo se pueden obtener resultados como:
 
 * hello world, upsi hello world, upsi 3 1 hello world, upsi 2 ḧello  *--- ḧello  *--- 
 * 
 * ḧello  *---
 

In [0]:
#include <fcntl.h>           /* For O_* constants */
#include <sys/stat.h>        /* For mode constants */
#include <semaphore.h>
#include <iostream>
#include <pthread.h>
#include <unistd.h>
#include <stdio.h>
#define SUCCESS 0
#define THREADS 3
#define NUM_CLIENTS_SAME_TIME 1
#define forn(x) for (int i =0; i< x; i++)

#define SNAME "/mysem2"

/*
 *  Run with the flag -pthread
 */

using namespace std;

sem_t *sem;

void * function (void *ap){
    sem_wait(sem);
    double val= 0;
    int code =  *(int*)ap ;
    cout << "hello world, upsi " << code << " ";
    fflush(stdout);
    sleep(1);
    cout << "ḧello  *--- "  << endl;
    sem_post(sem);
}

void create_threads(){
    pthread_t hilo[THREADS];
    int *values = new int[THREADS];
    for (int i=1; i<=THREADS; i++)
        values[i-1] = i;

    int result;
    int arg =1,c; /*  c -> control variable */

    for (int i =0; i< THREADS; i++)
        result = pthread_create(&hilo[i], NULL,  function, (void*) &values[i]);
    for (int i =0; i< THREADS; i++)
        result = pthread_join (hilo[i], NULL); //Wait until the threads finish
}


int main (){
    sem = sem_open(SNAME, O_CREAT, 0644, NUM_CLIENTS_SAME_TIME );
    create_threads();
    sem_close(sem);
    sem_unlink(SNAME);
}

##Mutex

Éste método de sincronización, consiste en bloquear el acceso a la zona crítica a los demás procesos, mientras un solo proceso está haciendo ejecuciones en esa sección.

El mutex, es una **variable atómica** por lo que siempre se ejecutará en el mismo orden y no será dividida. En si, mutex es una variable binaria, donde 0 es bloqueado y 1 desbloqueado.

Podemos verlo en el siguiente ejempo hecho en C:

In [0]:
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <string.h> 
void *myfunc1(void *ptr);
void *myfunc2(void *ptr);
pthread_mutex_t lock; /*mutex*/
int a[100];
int main(){
    pthread_t thrd1, thrd2;
    int thret1, thret2;
    char *msg1 = "primer hilo";
    char *msg2 = "segundo hilo";
    memset(a, 0, sizeof(a));
    thret1 = pthread_create(&thrd1, NULL, myfunc1, (void *)msg1);
    thret1 = pthread_create(&thrd1, NULL, myfunc2, (void *)msg1);
    pthread_join(thrd1, NULL);
    pthread_join(thrd2, NULL);
    printf("thret1 = %d\n", thret1);
    printf("thret2 = %d\n", thret2);
    return 0;
}
void *myfunc1(void *ptr){/*funcion que escribe*/
    int i;
    char *msg = (char *)ptr;
    printf("msg: %s\n", msg);
    pthread_mutex_lock(&lock);
    for(i = 0; i < 100; i++){
        printf("X");
        a[i] = i;
    }
    pthread_mutex_unlock(&lock);
}
void *myfunc2(void *ptr){/*funcion que lee*/
    int i;
    char *msg = (char *)ptr;
    printf("msg: %s\n", msg);
    pthread_mutex_lock(&lock);
    for(i = 0; i < 100; i++){
        printf("%d, ",a[i]);
    }
    pthread_mutex_unlock(&lock);
}

En el código anterior, podemos ver que se declara la variable mutex, y una variable global a que es un arreglo, en la cual se harán cambios y se usará como la variable compartida en la que podŕian haber problemas.

El programa tiene dos funciones: una que escribe y una que lee, se pone la sentencia *pthread_mutex_lock* dándole como argumento el mutex delcarado arriba, para que bloquee el código que le sigue abajo a los demás procesos. A l finalizar la zona crítica, en éste caso los ciclos en la sdos funciones, se libera la zona con *pthread_mutex_unlock(&lock);*, haciendo aśi que se sincronice el uso del recurso (el arreglo a global) y no se corrompan datos.

##Pipes o tubos
Los pipes son métodos de sincronización basados en el paso de mensajes. Consiste en la apertura de un canal vía memoria, en donde un proceso escribe y el otro al otro lado lee. Los pipes sólo pueden ser usados desde un proceso padre hacia los procesos hijo. El pipe usa descriptores, los cuales son variables que controlan el flujo de mensajes en el pipe.

Cada pipe tiene en cada extremo dos funciones: escribir y leer.
En el momento en el que uno de los procesos a un extremo lee, el descriptor de escritura de bloquea, y en el otro estremo el descriptor de lectura es bloqueado mientras el de escritura es habiliado. En ese orden de ideas **solo se puede dar una única lectura y una única escritura en el pipe**.

En el caso en el que se llene el pipe, se bloque al descriptor de escritura del autor que envía, mientras el de lectura va despachando.

Al acabar el paso de mensajes por el pipe, el pipe es destruído por el sistema operativo.

A continuación un ejemplo en C, en donde un proceso padre hace fork y crea un proceso hijo, para luego enviarle un mensaje por un buffer en el pipe de *Hola padre*

In [0]:
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <pthread.h>
/*clang pipes.c -pthread -o pipout*/
/*./pipout */
int main()
{
    int lectura = 0;
    int escritura = 1;
    char mensaje[32] = "Hola padre"; /*mensaje a enviar por el pipe desde proceso padre a proceso hijo*/
    char buffer[32];
    int fd[2];
    int r = pipe(fd);/*creacion del pipe*/
    pid_t pid;
    pid = fork();/*subdivision del proceso padre en proceso hijo*/
    if(r == -1){
        perror("error en la creacion de la tuberia");
        exit(0);
    }
    if(pid < 0){
        perror("error en proceso creado");
    }
    if(pid == 0){/*PROCESO HIJO*/
        close(fd[lectura]);/*como el hijo no lee se bloquea la lectura de el*/
        printf("bloqueo lectura hijo \n");
        r = write(fd[escritura], mensaje, sizeof(mensaje)+1);/*envia el mensaje por el pipe*/
        printf("escritura hijo hecha \n");
        close(fd[escritura]);/*bloquea el lado de escritura*/
    }else{
        close(fd[escritura]);/*PROCESO PADRE*/
        printf("bloqueo escritura padre \n");
        r = read(fd[lectura], buffer, sizeof(buffer)+1);/*lee lo que le enviaron en el pipe*/
        printf("buffer cargado del pipe \n");
        printf("%s\n", buffer);/*imprime lo que llego del pipe*/
        close(fd[lectura]);/*cierra el lado de lectura*/
    }
return 0;
}


##Bloqueo de tablas
Éste método de bloqueo, se caracteriza por inhabilitar tablas para que estas no puedan ser accedidas por otra sesion. Suponiendo que un usuario desea obtener un turno y para ello el sistema le otorga el siguiente turno al que tiene registrado, para esto se debe aumentar el valor de ese turno y después retornar dicho número 

In [0]:

UPDATE contador SET valor = valor + 1;
SELECT valor FROM contador;


si dos sesiones ejecutan el procedimiento almacenado que contenga las instrucciones es posible que una sesión interfiera en la ejecución de la otra y se de que tanto la primera sesión como la segunda sesión aumenten el turno, y al final a ambas sesiones se les retorne el mismo resultado


In [0]:
UPDATE contador SET valor = valor + 1; -- pasa a 1
UPDATE contador SET valor = valor + 1; -- pasa a 2
SELECT valor FROM contador; -- retorna 2
SELECT valor FROM contador; -- retorna 2

una de las soluciones es cada sesion bloque la tabla contador y la desbloque hasta que le sea retornado el valor, como se muestra a continuacion:

In [0]:
LOCK TABLES contador WRITE;
UPDATE contador SET valor = valor + 1; -- incrementar el contador
SELECT valor FROM contador; -- retornar el contador asignado
UNLOCK TABLES;

Realizado por:
* Nicolas Ricardo Enciso
* Diego Felipe Rodriguez Chaparro
* Camilo Alfonso Mosquera Benavides

Lenguajes de Programación,  2018-1